In [ ]:
import os, json
import matplotlib.pyplot as plt
import pprint


# root = '/private/home/delong/workspace/subobjects-VLM/runs/clevr_caption/SmolLM2-135M-Instruct-vae(384px)'
# splits = ['train', 'val']

# root = '/private/home/delong/workspace/subobjects-VLM/runs/clevr_caption/SmolLM2-135M-Instruct-dinov2_small(384px)'
# splits = ['train', 'val']

# root = '/private/home/delong/workspace/subobjects-VLM/runs/pixmo_cap/Llama-3_2-1B-dinov2_small(768px)'
# splits = ['train', 'val']

# root = '/private/home/delong/workspace/subobjects-VLM/runs/imagenet/SmolLM2-135M-Instruct-dinov2_small(768px)'
# splits = ['train', 'val']

root = '/private/home/delong/workspace/subobjects-VLM/runs/sharegpt4v/Llama-3_2-1B-dinov2_small(768px)'
splits = ['share-captioner_coco_lcs_sam_1246k_1107.json', 'sharegpt4v_instruct_gpt4-vision_cap100k.json']

excluding = []



tokenizer_families = os.listdir(root)
all_results = {}
for tokenizer_family in tokenizer_families:
    all_results[tokenizer_family] = []
    runs = os.listdir(os.path.join(root, tokenizer_family))
    for run in runs: 
        path = os.path.join(root, tokenizer_family, run, 'runs')
        if run in excluding or not os.path.exists(path):
            continue
        files = os.listdir(path)
        for file in files:
            if file.startswith('vlm_eval') and file.endswith('.json'):
                with open(os.path.join(path, file), 'r') as f:
                    data = json.load(f)
                    all_results[tokenizer_family].append(data)
print(len(all_results))


In [28]:
def sort_results(results):

    def sort_key(result):
        visual_tokenizer_config = result['checkpoint_args']['visual_tokenizer_config']
        if '/patch' in visual_tokenizer_config:
            return f"{int(visual_tokenizer_config.split('patch_')[-1].split('_')[0]):2}"
        else:
            return visual_tokenizer_config + f"{result['checkpoint_args']['max_visual_tokens']:3}"

    # Sort the results based on the visual_tokenizer_config
    results = sorted(results, key=sort_key)
    return results

for tokenizer_family in all_results.keys():
    all_results[tokenizer_family] = sort_results(all_results[tokenizer_family])

In [31]:
for split in splits:

    for tokenizer_family, results in all_results.items():
        
        for i, result in enumerate(results):
            if result['evaluation_args']['split'] != split:
                continue

            if not result['evaluation_finished']:
                print(f"Skipping {result['evaluation_args']} because it didn't finish")
                continue

            print(result.keys())

In [ ]:


color_offset = 15

for split in splits:

    plt.figure(figsize=(20, 10))
    cmaps = ['Greys', 'Blues', 'Reds', 'Greens']

    for tokenizer_family, results in all_results.items():
        colors = plt.cm.get_cmap(cmaps.pop(0), len(results)+color_offset)

        previous_x, previous_y = None, None

        for i, result in enumerate(results):
            if result['evaluation_args']['split'] != split:
                continue

            visual_tokenizer_config = result['checkpoint_args']['visual_tokenizer_config']
            tokenizer_name = visual_tokenizer_config.split('/')[-1].replace('.json', '')
            tokenizer_name = f"{tokenizer_name} ({result['checkpoint_args']['max_visual_tokens']})"

            plt.scatter(result['average_visual_tokens'], result['average_loss'], label=tokenizer_name, color=colors(i+color_offset), s=100)
            plt.text(result['average_visual_tokens'], result['average_loss'], f'{tokenizer_name}', color=colors(i+color_offset))

            if previous_x is not None:
                plt.plot([previous_x, result['average_visual_tokens']], [previous_y, result['average_loss']], color=colors(i+color_offset))

            previous_x, previous_y = result['average_visual_tokens'], result['average_loss']

    plt.title(split)
    # plt.legend(loc='lower right')
    # make x axis log scale
    # plt.xscale('log')
    plt.show()
